# GradCAM Activation Map Visualization

In [ ]:
!python cam.py --image-path "./test_play_1stframe.png" --use-cuda

<br>

<br>

<br>

# Count number of parameters

### Import libraries

In [ ]:
import os
os.getcwd()

In [ ]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models, transforms, utils
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc
from PIL import Image
import json
%matplotlib inline

import pytorchvideo
from torchvision.datasets.video_utils import VideoClips
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler, TensorDataset
# from torchsampler import ImbalancedDatasetSampler

# import ResNet

import cv2

import pandas as pd
import numpy as np
import os
import time
import pickle
import json
import random
import urllib
import copy
import matplotlib.pyplot as plt

from data_loader import VideoDataset
# from Utils import do_epoch
from Utils.metric import *
# from Utils.utility import *

import myutils

from torchvideotransforms import video_transforms, volume_transforms


from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    ShortSideScale,
    UniformTemporalSubsample,
    AugMix,
    create_video_transform
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    ToTensor
)

from prettytable import PrettyTable

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

### Load the trained model

In [ ]:
class Attention(nn.Module):
    def __init__(self, feature_dim):
        super(Attention, self).__init__()

        self.attn_1 = nn.Linear(feature_dim, feature_dim)
        self.attn_2 = nn.Linear(feature_dim, 1)

        # inititalize
        nn.init.xavier_uniform_(self.attn_1.weight)
        nn.init.xavier_uniform_(self.attn_2.weight)
        self.attn_1.bias.data.fill_(0.0)
        self.attn_2.bias.data.fill_(0.0)

    def forward(self, x):
        """
        Input x is encoder output
        return_attention decides whether to return
        attention scores over the hidden states
        """
#         frame_length = x.shape[1]

        self_attention_scores = self.attn_2(torch.tanh(self.attn_1(x)))
        
        return self_attention_scores

In [ ]:
# need to change this class if want to load the trained model with attention module
class Resnt34LSTM(nn.Module):
    def __init__(self, params_model):
        super(Resnt34LSTM, self).__init__()
        num_classes = 2
        pretrained = True
        self.rnn_hidden_size = 60
        rnn_num_layers = 1
        
        baseModel = models.resnet34(pretrained=pretrained)  
        num_features = baseModel.fc.in_features
        baseModel.avgpool = nn.AdaptiveMaxPool2d(output_size=(1,1))
        baseModel.fc = Identity()
        self.baseModel = baseModel
        
        self.rnn = nn.LSTM(num_features, self.rnn_hidden_size, rnn_num_layers)
        self.fc1 = nn.Linear(self.rnn_hidden_size*20, num_classes)

        
    def forward(self, x):        
        b_z, ts, c, h, w = x.shape
        
        
        ii = 0
        y = self.baseModel((x[:,ii]))
        output, (hn, cn) = self.rnn(y.unsqueeze(0))
        for ii in range(1, ts):
            y = self.baseModel((x[:,ii]))
            out, (hn, cn) = self.rnn(y.unsqueeze(0), (hn, cn))
            output = torch.cat((output.view(b_z,-1,self.rnn_hidden_size), out.view(b_z,-1,self.rnn_hidden_size)), dim=1)

        
        final_out = self.fc1(output.view(b_z,-1)) 
        
        return final_out 
    
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x    

In [ ]:
params_model={
        "num_classes": 2,
#         "dr_rate": 0.2,
        "pretrained" : True,
        "rnn_num_layers": 1,
        "rnn_hidden_size": 60,
        "attention": False}
model = Resnt34LSTM(params_model)  
model = model.to(device)

# criterion / loss function
criterion = nn.CrossEntropyLoss()

checkpoint = torch.load("D:/0_Graduation Thesis/models/weights/Resnet34_LSTM60_MAXpool_WS_OS_allHS_20220530_best_accuracy.pt")
model.load_state_dict(checkpoint)
model.eval()

### Counting starts here

In [ ]:
# counting function

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [ ]:
count_parameters(model)

In [ ]:
# rn50 = models.resnet50(pretrained=True)
# count_parameters(rn50)

In [ ]:
# counting sum in one line
sum(dict((p.data_ptr(), p.numel()) for p in model.parameters()).values())